In [3]:
import pandas as pd

In [4]:
consrv_big = pd.read_csv("Conservative.csv")

In [5]:
consrv_small = pd.read_csv("Conservatives.csv")

In [6]:
republic = pd.read_csv("Republican.csv")

In [7]:
#cleaning big_consv first :P
consrv_big_score_sorted = republic.sort_values(by='score',ascending=False)
consrv_big_score_sorted_2k = consrv_big_score_sorted[:2000]

In [8]:
consrv_big_score_sorted_2k

,Unnamed: 0,comment,score,type
170,170,Breaking News: Amy Coney-Barrett says she spea...,1062,0
171,171,"Since Amy Coney Barrett is alive, can she fair...",742,0
379,379,This letter is literally one of the most despi...,698,0
1412,1412,"It’s because r/news , r/reddit, r/Christianity...",625,0
1123,1123,"Gay here. Haven’t lost any rights yet, but I’m...",601,0
...,...,...,...,...
6127,6127,"Wait, I'm confused. Is Trump a diabolical geni...",32,0
6903,6903,Knife fights are “normal” and “healthy” for te...,32,0
2764,2764,As a Wisconsin resident its frustrating being ...,32,0
3047,3047,I’m scared at how they think it’s ok to shoot ...,32,0


In [9]:
#pip install clean-text
#pip install unidecode

In [10]:
from cleantext import clean

In [11]:
#reference
'''clean("some input",
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
    no_urls=False,                  # replace all URLs with a special token
    no_emails=False,                # replace all email addresses with a special token
    no_phone_numbers=False,         # replace all phone numbers with a special token
    no_numbers=False,               # replace all numbers with a special token
    no_digits=False,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=False,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_phone_number="<PHONE>",
    replace_with_number="<NUMBER>",
    replace_with_digit="0",
    replace_with_currency_symbol="<CUR>",
    lang="en"                       # set to 'de' for German special handling
) '''

'clean("some input",\n    fix_unicode=True,               # fix various unicode errors\n    to_ascii=True,                  # transliterate to closest ASCII representation\n    lower=True,                     # lowercase text\n    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them\n    no_urls=False,                  # replace all URLs with a special token\n    no_emails=False,                # replace all email addresses with a special token\n    no_phone_numbers=False,         # replace all phone numbers with a special token\n    no_numbers=False,               # replace all numbers with a special token\n    no_digits=False,                # replace all digits with a special token\n    no_currency_symbols=False,      # replace all currency symbols with a special token\n    no_punct=False,                 # remove punctuations\n    replace_with_punct="",          # instead of removing punctuations you may replace them\n    replace_with_url="<

In [12]:
#lower casing and unicode fix
cleaned_comments = []
for comment in consrv_big_score_sorted_2k.comment :
    tempdict = {}
    clean_comment = clean(comment,
    fix_unicode=True,               # fix various unicode errors
    lower=True,                     # lowercase text
    no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=True,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=True,                # replace all digits with a special token
    no_currency_symbols=True,
    no_punct=True,                      # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_digit="",
    replace_with_currency_symbol="",
    lang="en")
    tempdict["comment"]=clean_comment
    cleaned_comments.append(tempdict)

In [13]:
clean_1_df = pd.DataFrame(cleaned_comments)

In [14]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [16]:
import re
clean_1_df['comment'] = clean_1_df['comment'].apply(remove_emoji)

In [17]:
from bs4 import BeautifulSoup
import requests, lxml

def spellcorrect(word):
    headers = {
        'User-agent':
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
    }

    params = {
      'q': word,
      'hl': 'en',
      'gl': 'us',
    }

    html = requests.get('https://www.google.com/search?q=', headers=headers, params=params).text
    soup = BeautifulSoup(html, 'lxml')
    try :
      corrected_word = soup.select_one('a.gL9Hy').text

      corrected_word_link = f"https://www.google.com{soup.select_one('a.gL9Hy')['href']}"
      search_instead_for = soup.select_one('a.spell_orig').text
      search_instead_for_link = f"https://www.google.com{soup.select_one('a.spell_orig')['href']}"
      print(f'{corrected_word}')
    except:
      pass

In [101]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [102]:
clean_1_df_with_stop_words = clean_1_df.copy()
clean_1_df['comment'] = clean_1_df['comment'].apply(stopwords)

In [103]:
from collections import Counter
cnt = Counter()
for text in clean_1_df['comment'].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(20)

[('people', 410),
 ('like', 313),
 ('dont', 270),
 ('trump', 244),
 ('im', 185),
 ('would', 176),
 ('think', 174),
 ('get', 163),
 ('one', 156),
 ('us', 151),
 ('see', 139),
 ('biden', 130),
 ('left', 127),
 ('right', 126),
 ('even', 118),
 ('democrats', 117),
 ('know', 114),
 ('want', 107),
 ('going', 105),
 ('time', 104)]

In [104]:
#from textblob import TextBlob
#clean_1_df['comment'] = clean_1_df['comment'].apply(lambda x: str(TextBlob(x).correct()))

In [105]:
'''Lemmatization is the process of converting a word to its base form. 
The difference between stemming and lemmatization is, 
lemmatization considers the context and converts the word to its meaningful base form,
whereas stemming just removes the last few characters, often leading to incorrect
meanings and spelling errors. Here, lemmatization only performed. We need to provide 
the POS tag of the word along with the word for lemmatizer in NLTK. Depending on the POS, 
the lemmatizer may return different results.'''

'Lemmatization is the process of converting a word to its base form. \nThe difference between stemming and lemmatization is, \nlemmatization considers the context and converts the word to its meaningful base form,\nwhereas stemming just removes the last few characters, often leading to incorrect\nmeanings and spelling errors. Here, lemmatization only performed. We need to provide \nthe POS tag of the word along with the word for lemmatizer in NLTK. Depending on the POS, \nthe lemmatizer may return different results.'

In [106]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV} # Pos tag, used Noun, Verb, Adjective and Adverb
# Function for lemmatization using POS tag
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])
clean_1_df['comment'] = clean_1_df['comment'].apply(lemmatize_words)

In [107]:
#tokenisization

#Creating function for tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text
# Passing the function to 'text_rare' and store into'text_token'
clean_1_df['comment'] = clean_1_df['comment'].apply(lambda x: tokenization(x.lower()))
clean_1_df['comment'].head()

0    [break, news, amy, coneybarrett, say, speaks, ...
1    [since, amy, coney, barrett, alive, fairly, ru...
2    [letter, literally, one, despicable, thing, iv...
3    [rnews, rreddit, rchristianity, pretty, much, ...
4    [gay, havent, lose, right, yet, im, really, en...
Name: comment, dtype: object

In [108]:
#clean_1_df.to_csv("conservative-big-2k-toekns.csv")

In [109]:
#clean_1_df.to_csv("conservative-small-2k-toekns.csv")

In [110]:
#clean_1_df.to_csv("republic-2k-toekns.csv")

In [30]:
from bs4 import BeautifulSoup
import requests, lxml

def spellcorrect(word):
    headers = {
        'User-agent':
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
    }

    params = {
      'q': word,
      'hl': 'en',
      'gl': 'us',
    }

    html = requests.get('https://www.google.com/search?q=', headers=headers, params=params).text
    soup = BeautifulSoup(html, 'lxml')
    try :
          corrected_word = soup.select_one('a.gL9Hy').text

          corrected_word_link = f"https://www.google.com{soup.select_one('a.gL9Hy')['href']}"
          search_instead_for = soup.select_one('a.spell_orig').text
          search_instead_for_link = f"https://www.google.com{soup.select_one('a.spell_orig')['href']}"
          
    except:
          return word


In [31]:
comment_list = []
for comment in clean_1_df['comment'].head(5):
  
  com = spellcorrect(comment)
  comment_list.append(com)

In [32]:
comment_list

['breaking news amy coneybarrett says she speaks but could fairly judge a case on free speech',
 'since amy coney barrett is alive can she fairly rule on a capital murder case',
 "this letter is literally one of the most despicable things i've ever seen",
 'its because rnews reddit christianity and pretty much everywhere else is democrats',
 'gay here havent lost any rights yet but im really enjoying my lower taxes and my rent hasnt gone up in years']